In [25]:
from sklearn.metrics import  accuracy_score
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [26]:
train_size = 0.6
test_size = 0.4

In [27]:
data = load_iris()

In [28]:
dataset = data['data']
target = data['target'].reshape(-1,1)

In [29]:
final_data = np.hstack((dataset,target))
np.random.shuffle(final_data)

In [30]:
pd_data = pd.DataFrame(final_data)
pd_data.columns= ['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)','target']
pd_data = pd_data[(pd_data['target']==0) | (pd_data['target']==1)]
pd_data.tail(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
148,6.5,2.8,4.6,1.5,1.0
149,5.1,2.5,3.0,1.1,1.0


In [31]:
final_data = pd_data.values

train_data = final_data[:int(len(final_data)*train_size)]
test_data = final_data[:int(len(final_data)*test_size)]

train_data.shape,test_data.shape

((60, 5), (40, 5))

In [32]:
X_train = train_data[:,:-1]
y_train = train_data[:,-1:]


X_test = test_data[:,:-1]
y_test = test_data[:,-1:]


X_train.shape,y_train.shape,X_test.shape,y_test.shape

((60, 4), (60, 1), (40, 4), (40, 1))

In [33]:
def gradient(y,y_pred,x):
    return np.dot(x.T,(y-y_pred))

In [43]:
def newton_grad(y,y_pred,x):
    return np.dot(np.linalg.inv(np.dot(x.T,x)),gradient(y,y_pred,x))

In [44]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [45]:
def cost(y,y_pred):
    cost = -np.mean((y*np.log(y_pred))+((1-y)*(np.log(1-y_pred))))
    return cost

In [46]:
dummy_train = np.hstack((np.ones((len(train_data),1)),X_train))
weight = np.zeros((len(dummy_train[0]),1))
lr = 0.001
epoch = 1000


for i in range(epoch):
    y_pred = sigmoid(np.dot(dummy_train,weight))
    co = cost(y_train,y_pred)
    gra = gradient(y_train,y_pred,dummy_train)
    ne_grad = newton_grad(y_train,y_pred,dummy_train)
    
#     weight += lr*gra
    weight += ne_grad
    
    
    if i%100 == 0:
        print('Epoch : ',i,' Loss:',co)

Epoch :  0  Loss: 0.6931471805599454
Epoch :  100  Loss: 0.012921213939278928
Epoch :  200  Loss: 0.006691733739430968
Epoch :  300  Loss: 0.004552889697823658
Epoch :  400  Loss: 0.0034640410029047183
Epoch :  500  Loss: 0.0028020782276353844
Epoch :  600  Loss: 0.0023561508907381985
Epoch :  700  Loss: 0.0020348729001402376
Epoch :  800  Loss: 0.0017921345335916003
Epoch :  900  Loss: 0.0016021225538406397


In [47]:
weight

array([[-2.55114091],
       [-0.91620106],
       [-1.88104342],
       [ 4.06245332],
       [ 2.63006928]])

In [48]:
def predict(x,w):
    sig = sigmoid(np.dot(x,w[1:])+w[0])
    sig[sig > 0.5] = 1
    sig[sig < 0.5] = 0
    return sig

In [49]:
accuracy_score(predict(X_test,weight),y_test)

1.0

In [50]:
class logisticregression():
    def __init__(self,train_data,train_labels,lr=0.01,batch_size=None,epoch=10):
        dummy_once = np.ones((len(train_data),1))
        self.train_data = np.hstack((dummy_once,train_data))
        self.train_labels = train_labels
        
        self.params = np.zeros((len(self.train_data[0]),1))
        
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def cost(self,y,y_pred):
        return -np.mean(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))
    
    def gradient(self,y,y_pred,x):
        return np.dot(x.T,(y_pred-y))
    
    def train(self):
        for i in range(self.epoch):
            y_pred = self.sigmoid(np.dot(self.train_data,self.params))
            loss = self.cost(self.train_labels,y_pred)
            
            gra = self.gradient(self.train_labels,y_pred,self.train_data)
            
            self.params -= self.lr*gra
            
            if i%10 == 0 or i == self.epoch-1:
                print('Epoch : {}  Loss: {}'.format(i,loss))
    def predict(self,test_data):
        result = sigmoid(np.dot(test_data,self.params[1:])+self.params[0])
        result[result >= 0.5 ] = 1
        result[result < 0.5 ] = 0
        return result
    
    def evaluate(self,test_data,labels):
        accuracy = accuracy_score(self.predict(test_data),labels)
        return accuracy

In [51]:
logistic = logisticregression(X_train,y_train,epoch=10)

In [52]:
logistic.train()

Epoch : 0  Loss: 0.6931471805599454
Epoch : 9  Loss: 0.8527789280483123


In [53]:
logistic.evaluate(X_test,y_test)

0.775